<a href="https://colab.research.google.com/github/raldebsi/test-public-repo-1/blob/master/TitleGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets transformers

import transformers
from datasets import load_dataset, load_metric

data = load_dataset("json", data_files="/content/clean_wiki.json")
print(data)

# import transformers
# from datasets import load_dataset, load_metric

# data = load_dataset("json", data_files="/content/clean_wiki.json")
# # Assuming your JSON file has a single key representing the entire dataset
# # Modify 'your_dataset_key' with the actual key
dataset_key = list(data.keys())[0]

# Now you can split the dataset
datasets_train_test = data[dataset_key].train_test_split(test_size=0.2)
datasets_train_validation = datasets_train_test["train"].train_test_split(test_size=0.1)

data[dataset_key] = datasets_train_validation["train"]
data["validation"] = datasets_train_validation["test"]
data["test"] = datasets_train_test["test"]

print(data)

from transformers import AutoTokenizer

model_checkpoint = "t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

import transformers
from datasets import load_dataset

# Prepare a list to hold tokenized data
tokenized_data = []
max_length = 512
# Iterate over the 'train' split of the dataset
for item in data['train']:
    title = item['title']
    references = item['references']

    # Tokenize the title and references
    tokenized_title = tokenizer(title, return_tensors='pt', truncation=True, padding="max_length")
    tokenized_references = tokenizer(references, return_tensors='pt', truncation=True, padding="max_length")

    # Append the tokenized data
    tokenized_data.append({
        'title': tokenized_title,
        'references': tokenized_references
    })


from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments

# Load the pre-trained model
model = T5ForConditionalGeneration.from_pretrained("t5-base")

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=1,  # Reduced batch size
    per_device_eval_batch_size=1,  # Reduced batch size
    num_train_epochs=3,
    weight_decay=0.01,
    gradient_accumulation_steps=2,
    gradient_checkpointing = True# Added gradient accumulation
)

# Define a function to preprocess the data for the model
def preprocess_function(examples):
    # Tokenize the inputs (references) and labels (titles)
    inputs = tokenizer(examples['references'], truncation=True, padding="max_length", max_length=max_length)
    labels = tokenizer(examples['title'], truncation=True, padding="max_length", max_length=max_length)

    # The 'labels' should be used by the model to compute the loss
    inputs['labels'] = labels['input_ids']
    return inputs



tokenized_datasets = data.map(preprocess_function, batched=True)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer,
)

# Train the model
trainer.train()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['title', 'references'],
        num_rows: 982
    })
})
DatasetDict({
    train: Dataset({
        features: ['title', 'references'],
        num_rows: 706
    })
    validation: Dataset({
        features: ['title', 'references'],
        num_rows: 79
    })
    test: Dataset({
        features: ['title', 'references'],
        num_rows: 197
    })
})


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

...

Map:   0%|          | 0/197 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Epoch,Training Loss,Validation Loss
1,No log,0.028541
2,0.419400,0.023707
3,0.024500,0.023108


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TrainOutput(global_step=1059, training_loss=0.21087620695094325, metrics={'train_runtime': 1127.1983, 'train_samples_per_second': 1.879, 'train_steps_per_second': 0.939, 'total_flos': 1289772812206080.0, 'train_loss': 0.21087620695094325, 'epoch': 3.0})

In [6]:
# Evaluate the model
eval_results = trainer.evaluate(eval_dataset=tokenized_datasets['test'])

# Print the evaluation results
print("Evaluation results:", eval_results)


Evaluation results: {'eval_loss': 0.02166072279214859, 'eval_runtime': 19.3575, 'eval_samples_per_second': 10.177, 'eval_steps_per_second': 10.177, 'epoch': 3.0}


In [7]:
# Save the model and tokenizer
model.save_pretrained("./saved_model")
tokenizer.save_pretrained("./saved_model")


('./saved_model/tokenizer_config.json',
 './saved_model/special_tokens_map.json',
 './saved_model/spiece.model',
 './saved_model/added_tokens.json',
 './saved_model/tokenizer.json')

In [8]:
from transformers import T5ForConditionalGeneration, AutoTokenizer

# Load the saved model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("./saved_model")
tokenizer = AutoTokenizer.from_pretrained("./saved_model")


In [16]:
# Sample input reference
reference_text = "Armitage, David (2002). The Concepts of Atlantic History. In Armitage, David; Braddick, Michael J. (eds.). The British Atlantic World 1500–1800. David Eltis, et al. Atlas of the Transatlantic Slave Trade (2010)"

# Tokenize input
inputs = tokenizer(reference_text, return_tensors='pt', truncation=True, padding="max_length", max_length=512)

# Generate prediction
outputs = model.generate(
    input_ids=inputs['input_ids'],
    attention_mask=inputs['attention_mask'],
    min_length=5,  # Adjust as necessary
    num_beams=4,    # Adjust for different quality vs. speed trade-offs
    early_stopping=True
)

# Decode and print the generated title
generated_title = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated Title:", generated_title)


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Generated Title: Atlantic History of the British Atlantic


In [18]:
from google.colab import files

!zip -r saved_model.zip /content/saved_model
files.download("saved_model.zip")

  adding: content/saved_model/ (stored 0%)
  adding: content/saved_model/generation_config.json (deflated 30%)
  adding: content/saved_model/model.safetensors (deflated 9%)
  adding: content/saved_model/spiece.model (deflated 48%)
  adding: content/saved_model/config.json (deflated 63%)
  adding: content/saved_model/tokenizer_config.json (deflated 95%)
  adding: content/saved_model/tokenizer.json (deflated 74%)
  adding: content/saved_model/special_tokens_map.json (deflated 86%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>